In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pybaseball import pitching_stats, batting_stats, playerid_lookup, statcast

In [12]:
df = pd.read_csv(r"statcast_pitch_swing_data_20240402_20241030_with_arm_angle2.csv")

df = statcast(start_dt="2024-01-01", end_dt="2024-12-30")

This is a large query, it may take a moment to complete


c:\Users\graha\AppData\Local\Programs\Python\Python313\Lib\site-packages\pybaseball\statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)


Skipping offseason dates
Skipping offseason dates


  0%|          | 1/246 [00:02<08:19,  2.04s/it]c:\Users\graha\AppData\Local\Programs\Python\Python313\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
  1%|          | 2/246 [00:02<05:14,  1.29s/it]c:\Users\graha\AppData\Local\Programs\Python\Python313\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
  1%|          | 3/246 [00:03<03:23,  1.19it/s]c:\Users\graha\AppData\Local\Programs\Python\Python313\Lib\site-packages\pybaseball\datahelpers\postp

In [13]:
df['is_ball'] = df['zone'] > 10
df = df[df['game_type'] == 'R']
#lose abt 12k rows but I think its worth

# also want offset based on stand of batter

#removes pitchers with less than 500 pitches
pitches_per_pitcher = df.groupby(["pitcher", "pitch_type"])["pitch_number"].count().reset_index()
pitches_per_pitcher.columns = ["pitcher_id", "pitch_type", "total_pitches"]
pitchers_over_500 = pitches_per_pitcher[pitches_per_pitcher['total_pitches'] > 100][['pitcher_id', 'pitch_type']]
df = df.merge(pitchers_over_500, left_on=['pitcher', 'pitch_type'], right_on=['pitcher_id', 'pitch_type'], how='inner')

# gets average position X,Y of pitcher
running_avgs = ['release_pos_x','release_pos_z']
for column in running_avgs:
    df["running_avg_" + column] = (
        df.groupby(["pitcher",'pitch_type'])[column]
        .expanding()
        .mean()
        .shift()   # shift inside group
        .reset_index(level=[0,1], drop=True)
    )
    df = df.dropna(subset=["running_avg_" + column])
    df["offset_" + column] = df[column] - df["running_avg_" + column]


In [14]:
# Save the filtered DataFrame to a new CSV file
df.to_csv('filtered_pitchers_over_500.csv', index=False)

In [ ]:
df.sample(5)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches,is_ball,pitcher_id,running_avg_release_pos_x,offset_release_pos_x,running_avg_release_pos_z,offset_release_pos_z
334655,KN,2024-06-24,74.4,-1.48,5.51,"Waldron, Matt",682928,663362,field_out,hit_into_play,...,-17.17587,28.573695,38.866011,40.196244,False,663362,-1.150686,-0.329314,5.413627,0.096373
114417,FF,2024-08-27,100.2,-1.9,6.11,"Miller, Mason",664034,695243,NaN,swinging_strike,...,39.972518,33.382742,40.146423,6.324277,False,695243,-1.915823,0.015823,6.103228,0.006772
160550,FF,2024-08-14,93.6,-1.68,5.8,"Pagán, Emilio",571448,641941,field_out,hit_into_play,...,-17.461794,34.75878,31.67589,39.220226,False,641941,-1.646715,-0.033285,5.647080,0.15292
422585,FF,2024-05-30,91.2,2.48,5.96,"Harris, Hogan",650490,663687,NaN,ball,...,<NA>,<NA>,<NA>,<NA>,True,663687,2.337824,0.142176,5.952980,0.00702
193090,CU,2024-08-06,80.5,-1.13,6.45,"Birdsong, Hayden",696285,806185,NaN,foul,...,-14.797232,33.484841,40.877826,39.085504,False,806185,-1.122014,-0.007986,6.444722,0.005278
